In [1]:
!pip install spacy deepeval ragas datasets


  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached zipp-3.21.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached importlib_metadata-8.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached pluggy-1.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.1/559.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 45.2 MB/s eta 0:00:0000:01
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
Using cached i

In [2]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
from deepeval import evaluate
# from deepeval.metrics import EntityMatchMetric
# from ragas.metrics import contextual_recall, contextual_precision


ImportError: cannot import name 'EntityMatchMetric' from 'deepeval.metrics' (/Users/lilianahotsko/Desktop/University/Bachelors/LLMAnonymization/.venv/lib/python3.12/site-packages/deepeval/metrics/__init__.py)

In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 41.9 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
TRAIN_DATA = [
    ("Apple is looking at buying U.K. startup for $1 billion", 
     {"entities": [(0, 5, "ORG"), (27, 31, "GPE"), (44, 54, "MONEY")]}),
    
    ("San Francisco considers banning sidewalk delivery robots", 
     {"entities": [(0, 13, "GPE")]}),
    
    ("Tesla plans to open a new factory in Berlin", 
     {"entities": [(0, 5, "ORG"), (40, 46, "GPE")]}),
]

In [5]:
nlp = spacy.blank("en")

if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

/Users/lilianahotsko/Desktop/University/Bachelors/LLMAnonymization/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]


In [12]:
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()

    for epoch in range(25): 
        losses = {}
        for batch in minibatch(TRAIN_DATA, size=2):
            examples = []
            for text, annotations in batch:
                example = Example.from_dict(nlp.make_doc(text), annotations)
                examples.append(example)
            nlp.update(examples, drop=0.5, losses=losses)
        print(f"Losses at epoch {epoch}: {losses}")


Losses at epoch 0: {'ner': np.float32(24.125904)}
Losses at epoch 1: {'ner': np.float32(22.827713)}
Losses at epoch 2: {'ner': np.float32(20.997963)}
Losses at epoch 3: {'ner': np.float32(19.301352)}
Losses at epoch 4: {'ner': np.float32(15.878076)}
Losses at epoch 5: {'ner': np.float32(13.380622)}
Losses at epoch 6: {'ner': np.float32(11.051787)}
Losses at epoch 7: {'ner': np.float32(9.414019)}
Losses at epoch 8: {'ner': np.float32(8.062301)}
Losses at epoch 9: {'ner': np.float32(8.41878)}
Losses at epoch 10: {'ner': np.float32(7.807492)}
Losses at epoch 11: {'ner': np.float32(8.694886)}
Losses at epoch 12: {'ner': np.float32(8.147778)}
Losses at epoch 13: {'ner': np.float32(8.373519)}
Losses at epoch 14: {'ner': np.float32(8.040446)}
Losses at epoch 15: {'ner': np.float32(7.674814)}
Losses at epoch 16: {'ner': np.float32(7.6837277)}
Losses at epoch 17: {'ner': np.float32(7.0076714)}
Losses at epoch 18: {'ner': np.float32(6.6474457)}
Losses at epoch 19: {'ner': np.float32(7.045026)}
L

In [13]:
nlp.to_disk("custom_ner_model")

In [14]:
nlp_test = spacy.load("custom_ner_model")

EVAL_DATA = [
    {"text": "Apple is planning to acquire a startup in London", 
     "expected": [("Apple", "ORG"), ("London", "GPE")]},
    
    {"text": "Microsoft invested $10 million in OpenAI", 
     "expected": [("Microsoft", "ORG"), ("$10 million", "MONEY"), ("OpenAI", "ORG")]},
    
    {"text": "Elon Musk announced a new Tesla factory in Germany", 
     "expected": [("Elon Musk", "PERSON"), ("Tesla", "ORG"), ("Germany", "GPE")]}
]


In [ ]:
results = []
for example in EVAL_DATA:
    doc = nlp_test(example["text"])
    predicted_entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    metric = EntityMatchMetric()
    score = metric.evaluate(predicted_entities, example["expected"])
    
    results.append({"text": example["text"], "deepeval_score": score})


In [16]:
!pip install scikit-learn

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 47.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.6 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [18]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch
from sklearn.metrics import precision_score, recall_score, f1_score

In [19]:
# --- Step 3: Load Model and Prepare Evaluation Data ---
nlp_test = spacy.load("custom_ner_model")

EVAL_DATA = [
    {"text": "Apple is planning to acquire a startup in London", 
     "expected": [("Apple", "ORG"), ("London", "GPE")]},
    
    {"text": "Microsoft invested $10 million in OpenAI", 
     "expected": [("Microsoft", "ORG"), ("$10 million", "MONEY"), ("OpenAI", "ORG")]},
    
    {"text": "Elon Musk announced a new Tesla factory in Germany", 
     "expected": [("Elon Musk", "PERSON"), ("Tesla", "ORG"), ("Germany", "GPE")]}
]

# --- Step 4: Compute Precision, Recall, and F1-score ---
def evaluate_ner(nlp_model, eval_data):
    y_true = []
    y_pred = []

    for example in eval_data:
        doc = nlp_model(example["text"])
        predicted_entities = {(ent.text, ent.label_) for ent in doc.ents}
        expected_entities = set(example["expected"])

        for entity in expected_entities:
            y_true.append(1 if entity in expected_entities else 0)
            y_pred.append(1 if entity in predicted_entities else 0)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return precision, recall, f1

precision, recall, f1 = evaluate_ner(nlp_test, EVAL_DATA)

print("\n--- NER Evaluation Metrics ---")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")



--- NER Evaluation Metrics ---
Precision: 1.0000
Recall: 0.1250
F1-score: 0.2222


In [ ]:
# --- Step 5: Evaluate with Ragas ---
ground_truths = [ex["expected"] for ex in EVAL_DATA]
predictions = [[(ent.text, ent.label_) for ent in nlp_test(ex["text"]).ents] for ex in EVAL_DATA]

precision_score = contextual_precision(predictions, ground_truths)
recall_score = contextual_recall(predictions, ground_truths)

print("\n--- Ragas Metrics ---")
print(f"Contextual Precision: {precision_score}")
print(f"Contextual Recall: {recall_score}")

In [20]:
!pip install opik

  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached rpds_py-0.22.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.3 MB/s eta 0:00:00
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.6 MB/s eta 0:00:00
Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl (18 kB)
Using cached rpds_py-0.22.3-cp312-cp312-macosx_11_0_arm64.whl (342 kB)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
import os, openai
from opik import track

os.environ["OPIK_API_KEY"] = "6ZdPx21vK23sxyOqXzxlfHPsx" 
os.environ["OPIK_WORKSPACE"] = "naniiigock"

client = openai.OpenAI()

@track
def retrieve_context(input_text):
    return [
        "What specific information are you looking for?",
        "How can I assist you with your interests today?",
        "Are there any topics you'd like to explore or learn more about?",
    ]

@track
def generate_response(input_text, context):
    full_prompt = f'If the user asks a question that is not specific, use the context to provide a relevant response.\nContext: {", ".join(context)}\nUser: {input_text}\nAI:'
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=[{"role": "user", "content": full_prompt}]
    )
    return response.choices[0].message.content

context = retrieve_context("Hi how are you?")
print(generate_response("Hi how are you?", context))


OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/naniiigock/redirect/projects?name=Default%20Project.


I am just a virtual assistant, so I do not have feelings or emotions. How can I assist you today with your interests or any specific information you are looking for?


In [23]:
import opik
from opik.evaluation import evaluate_prompt
from opik.evaluation.metrics import Hallucination

os.environ["OPIK_API_KEY"] = "6ZdPx21vK23sxyOqXzxlfHPsx" 
os.environ["OPIK_WORKSPACE"] = "naniiigock"

# Create a dataset that contains the samples you want to evaluate
opik_client = opik.Opik()
dataset = opik_client.get_or_create_dataset("Evaluation test dataset")
dataset.insert([
    {"input": "Hello, world!", "expected_output": "Bonjour, monde!"},
    {"input": "What is the capital of France?", "expected_output": "Paris"},
])

# Run the evaluation
result = evaluate_prompt(
    dataset=dataset,
    messages=[{"role": "user", "content": "Translate the following text to French: {{input}}"}],
    model="gpt-3.5-turbo",  # or your preferred model
    scoring_metrics=[Hallucination()]
)


Evaluation: 100%|██████████| 3/3 [00:02<00:00,  1.13it/s]


╭─ Evaluation test dataset (3 samples) ─╮
│                                       │
│ Total time:        00:00:02           │
│ Number of samples: 3                  │
│                                       │
│ hallucination_metric: 0.0000 (avg)    │
│                                       │
╰───────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=338377;https://www.comet.com/opik/naniiigock/experiments/019512a9-e7f0-757f-9b45-e38676519aa5/compare?experiments=%5B%22019512aa-e419-710c-a75f-917d172d451c%22%5D\in your Opik dashboard]8;;\.

In [31]:
import spacy
from pathlib import Path
from spacy.util import load_model_from_path

model_path = Path("spacy/output/model-last")
nlp = load_model_from_path(model_path)

doc = nlp("test message")
for ent in doc.ents:
    print(f"{ent.text} -> {ent.label_}")


Maria -> GPE
Toronto -> GPE
